# Utils Script



### This script consists of the different utility functions for the  credit risk model development 
### The Functions do a range of tasks from 

    - Plot graphs 
    - Create extensive analytical tables 
    

In [6]:
import warnings
import datetime as dt
warnings.filterwarnings("ignore")

import re
import string 
import copy

import numpy as np
import pandas as pd



import matplotlib.pyplot as plt




print("Util Libraries successfully imported")

Util Libraries successfully imported


In [7]:
pd.__version__

'1.4.2'

In [9]:
# A function to calculate WOE and IV for numerical features
def woe_num(data, feature, gbflag):
    dt = pd.crosstab(index=data[feature], columns=data[gbflag])
    dt['Freq'] = dt.sum(axis=1)
    dt['Percentage'] = round((dt['Freq']/dt['Freq'].sum() *100),1)
    dt['% Good'] = round((dt['Good']/dt['Good'].sum() *100), 1)
    dt['% Bad'] = round((dt['Bad']/dt['Bad'].sum() *100), 1)
    dt['Bad Rate'] = round((dt['Bad']/dt['Freq']) *100, 1)
    dt['GoodBaddOdds'] = round(dt['Good']/dt['Bad'], 2)
    dt['WOE'] = np.log(dt['% Good']/dt['% Bad'])
    dt['IV'] = (dt['% Good']- dt['% Bad']) * dt['WOE']
#     dt['IV'] = ((dt['% Good']- dt['% Bad']) * dt['WOE']).sum()
   # dt = dt.sort_values(['WOE'])
    
    return dt
# Function to calculate WOE and IV for categorical variables
# Function to calculate WOE and IV for categorical variables
def woe_cat(data, feature, gbflag):
    dt = pd.crosstab(index=data[feature], columns=data[gbflag])
    dt['Freq'] = dt.sum(axis=1)
    dt['Proptn'] = dt['Freq']/dt['Freq'].sum()
    dt['% Good'] = dt['Good']/dt['Good'].sum() 
    dt['% Bad'] = dt['Bad']/dt['Bad'].sum()
    dt['Bad Rate'] = dt['Bad']/dt['Freq'] 
    dt['GoodBaddOdds'] = round(dt['Good']/dt['Bad'], 2)
    dt['WOE'] = np.log(dt['% Bad']/dt['% Good'])  # Ratio of Event to Non-Event
    dt['class IV'] = (dt['% Bad']-dt['% Good']) * dt['WOE']  # IV for each class/categpry
    dt['Variable IV'] = ((dt['% Bad']-dt['% Good']) * dt['WOE']).sum() #IV for the variable
    dt = dt.sort_values(['WOE'])
    
    return dt
# A function to get all variables WOE and IV
def woe_iv(df, variablie_list, GBFlag):   
    output = {}
    for variable in variablie_list:
        try:
            var = woe_cat(df, variable, GBFlag)
            
            for i in range(len(var.index.values)):
                output.setdefault('Variables',[]).append(variable)
                output.setdefault('Categories',[]).append(var.index.values[i])
                output.setdefault('WOE',[]).append(var['WOE'].values[i])
                output.setdefault('IV',[]).append(var['IV'].values[i])
        except Exception as e:
            pass

    return pd.DataFrame(output)
# woe_iv(data_universe_fai, data_universe_fai_cat.columns, 'GBFlag')
# A function that automates and return CA report
def Export_CA_Report(df, variablie_list, GBFlag, fileName='CA_Report'):
    
    # This inner function estimates WoE and IV which represent CA for all the provided variables
    def CA_Report(df, variablie_list, GBFlag):
        # Create a dictionary to track CA iteratively
        output = {}
        for variable in variablie_list:
            # use the function for estimating WoE for categorical variables to 
            var = woe_cat(df, variable, GBFlag)
            # append to dictionary
            output[variable]=var        

        return output
    
    
   # Instatiate above function to CA report for all provvided variables
    ca_result = CA_Report(df, variablie_list, GBFlag)
    # save CA report to Excel
    with pd.ExcelWriter(fileName +'.xlsx', engine="openpyxl") as writer:
        for variable_name, data in ca_result.items():
            data.to_excel(writer, sheet_name=variable_name)
    
    
def fill_binned_nan_values(df, variablie_list, GBFlag):
    for variable in variablie_list:
        if df[variable].isna().sum() > 0:
            # Run CA(WoE & IV) on variable
            run_ca = woe_num(df, variable, GBFlag)
            # get index value(bin class) of category with least woe
            bin_class = run_ca[run_ca['WOE']==max(run_ca['WOE'])].index.values[0]
            df[variable] = df[variable].fillna(bin_class)
        else:
            pass

In [5]:
pd.__version__

'1.4.2'

In [11]:
def missing_zero_values_table(df):
  
    """
    function for data audit, returns the count and percentage of missing and o values in each column
     %% parameters 

    df: [dataframe] 
    
    """
  
    zero_val = (df == 0.00).astype(int).sum(axis=0)
        
    zero_val_percent = (df == 0.00).astype(int).sum(axis=0) / len(df)
        
    mis_val = df.isnull().sum()
    
    mis_val_percent = round(df.isnull().sum() / len(df),1)
    
    mz_table = pd.concat([zero_val,zero_val_percent, mis_val, mis_val_percent], axis=1)
    
    mz_table = mz_table.rename(
                                columns = {0 : 'No of 0s', 1 : '% of 0s', 2: 'No of Missing Values', 3 : '% of Missing Values'})
    
    mz_table['Rows'] = len(df)
    mz_table['No of Unique'] = df.nunique()
    
    mz_table['Data Type'] = df.dtypes
    
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
        "There are " + str(mz_table.shape[0]) +
            " columns that have missing values.")
    
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
    return mz_table


def audit(data):
    
    result = {}
    
    for col in data.columns:
        No_unique = data[col].nunique()
        Missing = data[col].isna().sum()
        perc_missing = Missing/data.shape[0]
        Zeros = data[data[col] == 0].shape[0]
        perc_zeros = Zeros/data.shape[0]
        row_count = data[col].shape[0]
        
        result.setdefault('Variable', []).append(col)
        result.setdefault('No of Unique', []).append(No_unique)
        result.setdefault('No of Missing', []).append(Missing)
        result.setdefault('% Missing', []).append(perc_missing)
        result.setdefault('No of Zeros', []).append(Zeros)
        result.setdefault('% Zeros', []).append(perc_zeros)
        result.setdefault('No of Rows', []).append(row_count)
        
    res = pd.DataFrame(result)
    
    return res


In [12]:
from IPython.display import display_html


def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace(
        'table', 'table style="display:inline"'), raw=True)
    
    
def sample_first_prows(data, perc=0.8):
    '''function to get specified percentage of first rows in the dataframe
    '''
    return data.head(int(len(data)*(perc)))

# train = sample_first_prows(data)
# test = data.iloc[max(train.index):]


In [13]:
def fill_binned_nan_values(df, variablie_list, GBFlag):
    for variable in variablie_list:
        ## Coarse NaN group into the worst woe group if not more than 10% of total data shape
      
            # Run CA(WoE & IV) on variable using the woe_num function
            run_ca = woe_num(df, variable, GBFlag)
            # get index value(bin class) of category with worst woe category
            bin_class = run_ca[run_ca['WOE']==max(run_ca['WOE'])].index.values[0]
            df[variable] = df[variable].fillna(bin_class)
            
       

In [14]:
def replace_with_woe(df_with_selected_variables):
    '''
    This function replaces each category in the selected columns with
    their woe
    
    df_with_selected_variables :: Dataframe containing grouped/binned selected
    variables
    
    grpd_variables_woe_iv :: Dataframe containing WoE and IV of grouped/binned
    variables
    
    Note:: Make sure woe_iv function has returned it's result before running
    this function. result in this case - grpd_variables_woe_iv
    
    '''
    
    for variable in df_with_selected_variables.columns:
        for category in df_with_selected_variables[variable].unique():
            
            try:
                ### Get WoE from the dataframe running the WoE_IV function
                woe =variables_woe_iv[
                    (variables_woe_iv['Variables']==variable) & 
                    (variables_woe_iv['Categories']==category)
                ]['WOE'].values[0]

                ### Replace each category with their respective woe
                df_with_selected_variables[variable] = df_with_selected_variables[variable].replace(category, woe)
            except Exception as e:
                pass
    
    return df_with_selected_variables

In [15]:
# # A function to get all variables WOE and IV
def woe_iv(df, variablie_list, GBFlag):   
    output = {}
    for variable in variablie_list:
        try:
            var = woe_cat(df, variable, GBFlag)
            
            for i in range(len(var.index.values)):
                output.setdefault('Variables',[]).append(variable)
                output.setdefault('Categories',[]).append(var.index.values[i])
                output.setdefault('WOE',[]).append(var['WOE'].values[i])
                output.setdefault('Variable IV',[]).append(var['Variable IV'].values[i])
        except Exception as e:
            pass

    return pd.DataFrame(output)






# A function to get all variables WOE and IV
def woe_iv(df, variablie_list, GBFlag):   
    output = {}
    for variable in variablie_list:
        try:
            var =  (pd.crosstab(df[variable],df[GBFlag],normalize='columns').assign(woe=lambda dfx: np.log(dfx['Bad'] / dfx['Good'])).assign(iv=lambda dfx: np.sum(dfx['woe']*(dfx['Bad']-dfx['Good']))))
            
            for i in range(len(var.index.values)):
                output.setdefault('Variables',[]).append(variable)
                output.setdefault('Categories',[]).append(var.index.values[i])
                output.setdefault('WOE',[]).append(var['woe'].values[i])
                output.setdefault('Variable IV',[]).append(var['iv'].values[i])
        except Exception as e:
            pass

    return pd.DataFrame(output)

def iv(df, variablie_list, GBFlag):   
    output = {}
    for variable in variablie_list:
        try:
            var = woe_cat(df, variable, GBFlag)
            
            for i in range(len(var.index.values)):
                output.setdefault('Variables',[]).append(variable)
                output.setdefault('Variable IV',[]).append(var['Variable IV'].values[i])
        except Exception as e:
            pass

    return pd.DataFrame(output)

In [6]:
### This function helps in computing Gini from AUC


#https://stackoverflow.com/questions/66679372/how-does-the-predict-function-of-statsmodels-interact-with-roc-auc-score-of-scik


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
def compute_gini(y_test, y_train, test_preds, train_preds):
    # instantiate roc auc score from sci-kit learn metrics and compute AUC

    Test_AUC = roc_auc_score(y_test, test_preds)
    Train_AUC = roc_auc_score(y_train, train_preds)
    # Compute Gini from AUC
    Test_Gini = (2*Test_AUC) - 1
    Train_Gini = (2*Train_AUC) - 1
    return Test_Gini, Train_Gini

In [ ]:
def compute_recall(y_test, y_train, test_preds, train_preds):
    # instantiate roc auc score from sci-kit learn metrics and compute AUC
    train_preds = train_preds.to_list()
    train_preds = np.asarray(train_preds)
    test_preds = test_preds.to_list()
    test_preds = np.asarray(test_preds)
    Test_precision, Test_recall, thresh = precision_recall_curve(y_test, test_preds)
    Train_precision, Train_recall, train_thresh = precision_recall_curve(y_train, train_preds)

    return Test_recall, Train_recall

In [ ]:
def compute_precision(y_test, y_train, test_preds, train_preds):
    # instantiate roc auc score from sci-kit learn metrics and compute AUC
    train_preds = train_preds.to_list()
    train_preds = np.asarray(train_preds)
    test_preds = test_preds.to_list()
    test_preds = np.asarray(test_preds)
    Test_precision, Test_recall, thresh = precision_recall_curve(y_test, test_preds)
    Train_precision, Train_recall, train_thresh = precision_recall_curve(y_train, train_preds)

    return Test_precision, Train_precision

In [ ]:
def plot_by_woe(df_WoE, rotation_of_x_axis_labels = 0):
    x = np.array(df_WoE.index).astype(str)
    y = df_WoE['WOE']
    plt.figure(figsize = (26, 8))
    plt.plot(x, y, marker = 'o', linestyle = '--', color = 'k')
    #plt.xlabel(x)
    plt.ylabel('Weight of Evidence')
    plt.title(str(''))
    plt.xticks(rotation = rotation_of_x_axis_labels)